In [1]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from PIL import Image, ImageOps
import os

In [ ]:
np.set_printoptions(suppress=True)

general_model = keras.models.load_model("/Users/mm22/Documents/Development/twentypercentplus/models/dogcatperson/model/keras_model.h5", compile=False)
general_class_names = open("/Users/mm22/Documents/Development/twentypercentplus/models/dogcatperson/model/labels.txt", "r").readlines()
dog_model = keras.models.load_model('128dogs.h5')
cat_model = keras.models.load_model('catmodel.h5')

In [3]:
cat_categories = []
dog_categories = []

for root, dirs, files in os.walk("/Users/mm22/Downloads/Dog_images"):
    for dir_name in dirs:
        dog_categories.append(dir_name)
dog_categories

for root, dirs, files in os.walk("/Users/mm22/Downloads/cat_images"):
    for dir_name in dirs:
        cat_categories.append(dir_name)
cat_categories

['Sphynx',
 'Burmese',
 'Birman',
 'American Shorthair',
 'Egyptian Mau',
 'American Bobtail',
 'Ragdoll',
 'Abyssinian',
 'Siamese',
 'Maine Coon',
 'Bengal',
 'Scottish Fold',
 'Tuxedo',
 'British Shorthair',
 'Bombay',
 'Dilute Tortoiseshell',
 'Russian Blue',
 'Calico',
 'Persian',
 'Norwegian Forest Cat']

In [ ]:
def prepare_generalmodel(filepath):
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(filepath).convert("RGB")

    # resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array
    
    return data



def prepare_specificmodel(filepath):
    IMG_SIZE = 299
    img_array = cv2.imread(filepath)[...,::-1]
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
def second_largest(lst):
    # Initialize the largest and second largest variables and their indices
    largest = lst[0]
    largest_idx = 0
    second_largest = lst[1] if lst[1] != largest else float('-inf')
    second_largest_idx = 1 if lst[1] != largest else -1
    
    # Iterate through the list to find the largest and second largest elements and their indices
    for i in range(2, len(lst)):
        if lst[i] > largest:
            second_largest = largest
            second_largest_idx = largest_idx
            largest = lst[i]
            largest_idx = i
        elif lst[i] > second_largest and lst[i] != largest:
            second_largest = lst[i]
            second_largest_idx = i
    
    # Return the second largest element and its index
    return second_largest_idx


def third_largest(lst):
    # Initialize the three largest variables and their indices
    largest = lst[0]
    largest_idx = 0
    second_largest = lst[1] if lst[1] != largest else float('-inf')
    second_largest_idx = 1 if lst[1] != largest else -1
    third_largest = lst[2] if lst[2] not in (largest, second_largest) else float('-inf')
    third_largest_idx = 2 if lst[2] not in (largest, second_largest) else -1
    
    # Iterate through the list to find the three largest elements and their indices
    for i in range(2, len(lst)):
        if lst[i] > largest:
            third_largest = second_largest
            third_largest_idx = second_largest_idx
            second_largest = largest
            second_largest_idx = largest_idx
            largest = lst[i]
            largest_idx = i
        elif lst[i] > second_largest and lst[i] != largest:
            third_largest = second_largest
            third_largest_idx = second_largest_idx
            second_largest = lst[i]
            second_largest_idx = i
        elif lst[i] > third_largest and lst[i] not in (largest, second_largest):
            third_largest = lst[i]
            third_largest_idx = i
    
    # Return the third largest element and its index
    return third_largest_idx

In [ ]:
image_path = '/Users/mm22/Downloads/misc objects/image-30.jpg'

#general model
prediction = general_model.predict(prepare_generalmodel(image_path))
index = np.argmax(prediction)
class_name = general_class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print(prediction)
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

if class_name.strip()[2:] == "Person" and confidence_score > 0.8:
    print('Not a picture of a cat or dog')

elif class_name.strip()[2:] == "Cat":
    #specific model
    prediction = cat_model.predict([prepare_specificmodel(image_path)])
    #print(categories[int(prediction)])
    prediction_list = list(prediction[0])
    max_index = prediction_list.index(max(prediction_list))
    if max(prediction_list) > 0.9:
        print(max(prediction_list))
        print(cat_categories[max_index])
    else:
        second_index = second_largest(prediction_list)
        third_index = third_largest(prediction_list)
        print("Breed 1: ", cat_categories[max_index], max(prediction_list))
        print("Breed 2: ", cat_categories[second_index], prediction_list[second_index])
        print("Breed 3: ", cat_categories[third_index], prediction_list[third_index])
    
elif class_name.strip()[2:] == "Dog":
    #specific model
    prediction = dog_model.predict([prepare_specificmodel(image_path)])
    #print(categories[int(prediction)])
    prediction_list = list(prediction[0])
    max_index = prediction_list.index(max(prediction_list))
    if max(prediction_list) > 0.9:
            print(max(prediction_list))
            print(dog_categories[max_index])
    else:
        second_index = second_largest(prediction_list)
        third_index = third_largest(prediction_list)
        print("Breed 1: ", dog_categories[max_index], max(prediction_list))
        print("Breed 2: ", dog_categories[second_index], prediction_list[second_index])
        print("Breed 3: ", dog_categories[third_index], prediction_list[third_index])
else:
    print('Unknown image, please try again')

In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from PIL import Image, ImageOps
import os

np.set_printoptions(suppress=True)

# load in the path to your specfic model
dog_model = keras.models.load_model('')

# function to prepare the image
def prepare_specificmodel(filepath):
    IMG_SIZE = 299
    img_array = cv2.imread(filepath)[...,::-1]
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# get all the dog category names
dog_categories = []

for root, dirs, files in os.walk(""):
    for dir_name in dirs:
        dog_categories.append(dir_name)
dog_categories

image_path = ''
# model recognition

#specific model
prediction = dog_model.predict([prepare_specificmodel(image_path)])
#print(categories[int(prediction)])
prediction_list = list(prediction[0])
max_index = prediction_list.index(max(prediction_list))
print("Breed: ", dog_categories[max_index], max(prediction_list))